# SNAP - NEXUSFORGE - Browsing data

In this notebook, we will cover how to browse and traverse Entities.

### Setup

The token can be found in the upper right corner on https://bbp.epfl.ch/nexus/web/ and pasted here

In [1]:
import getpass
from bluepysnap.api import Api

BUCKET = "nse/test"
TOKEN = getpass.getpass()
api = Api(BUCKET, TOKEN)

 ·······································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································

### Browsing an entity

Let's first get an entity and show the available data:

In [2]:
entity = api.get_entity_by_id('https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/O0-20180419')
api.as_dataframe([entity], store_metadata=False).T

,0
id,https://bbp.epfl.ch/nexus/v1/resources/nse/tes...
type,DetailedCircuit
brainLocation.brainRegion.id,http://api.brain-map.org/api/v2/data/Structure...
brainLocation.brainRegion.label,Thalamus
circuitBase.type,DataDownload
circuitBase.url,file:///gpfs/bbp.cscs.ch/project/proj55/oreill...
circuitType,Microcircuit with O0 geometry
description,"Thalamus microcircuit v1, 2018 release"
edgeCollection.id,https://bbp.epfl.ch/nexus/v1/resources/nse/tes...
edgeCollection.type,EdgeCollection


All the values shown above can be accessed programmatically through the `Entity` object with the dot-notation:

In [3]:
print(f"""
Circuit:
-------------------------------------------------------
name: {entity.name}
description: {entity.description}
species: {entity.species.label}
region: {entity.brainLocation.brainRegion.label}
-------------------------------------------------------""")


Circuit:
-------------------------------------------------------
name: Thalamus microcircuit v1
description: Thalamus microcircuit v1, 2018 release
species: Mouse
region: Thalamus
-------------------------------------------------------


#### Access store metadata
There's a small caveat when accessing the store metadata values: instead of accessing them with the leading underscore, they are accessed without it.
For example the `entity` has the following metadata keys:

In [4]:
print(*sorted(set(entity.to_dict()) - set(entity.to_dict(store_metadata=False))), sep='\n')

_constrainedBy
_createdAt
_createdBy
_deprecated
_incoming
_outgoing
_project
_rev
_schemaProject
_self
_updatedAt
_updatedBy


To access them, we need to leave out the underscore. For example:

In [5]:
print(f"""
_createdAt: {entity.createdAt}
_createdBy: {entity.createdBy}
_project: {entity.project}
"""
)


_createdAt: 2019-10-02T13:39:20.493Z
_createdBy: https://bbp.epfl.ch/nexus/v1/realms/bbp/users/ivaska
_project: https://bbp.epfl.ch/nexus/v1/projects/nse/test



### Traversing an entity

While accessing the values with dot-notation is neat, it surely would be more awesome if there was a way to access the linked resources in the graph through the entity, right? 
Here's where traversing entities comes in to play.

In the former notebooks, we searched for a `MorphologyRelease` with a nexus id of a `DetailedCircuit`. The used search filters were:
```python
filters = {
    "^morphologyRelease": {
        "^memodelRelease": {
            "^nodeCollection": {
                "type": "DetailedCircuit"
            },
        },
    }}
```

While that was just an example of a "look-back" search, the problem remains: writing such a filter for a linear path is cumbersome. Fortunately, we can achieve the same by traversing the entity:

In [6]:
morphology_release_id = entity.nodeCollection.memodelRelease.morphologyRelease.id
morphology_release = api.get_entity_by_id(morphology_release_id)
api.as_dataframe([morphology_release], store_metadata=False).T

,0
id,https://bbp.epfl.ch/nexus/v1/resources/nse/tes...
type,MorphologyRelease
distribution.type,DataDownload
distribution.url,/gpfs/bbp.cscs.ch/project/proj55/iavarone/rele...
morphologyIndex.id,https://bbp.epfl.ch/nexus/v1/resources/nse/tes...
morphologyIndex.type,ModelReleaseIndex
name,O0-20180419_morph_release
wasAttributedTo,[{'id': 'https://bbp.epfl.ch/nexus/v1/resource...


Unfortunately, there is currently no way to traverse backwards with the use of dot-notation. 

I.e., we __can not__ go from morphology release back to the `DetailedCircuit` by doing something like:
```python
detailed_circuit_id = morphology_release.memodelRelease.nodeCollection.detailedCircuit.id
```